In [2]:
import pandas as pd
import numpy as np
import wrds
import matplotlib.pyplot as plt
import pickle as pkl
from dateutil.relativedelta import *


WRDS_DOWNLOAD_DIR = '/home/yu/OneDrive/CC/data/WRDS-download'

conn=wrds.Connection(wrds_username='xiaomowu')

# set sample date range
begdate = '01/01/2005'
enddate = '01/18/2021'

# set CRSP date range a bit wider to guarantee collecting all information
crsp_begdate = '01/01/2004'
crsp_enddate = '01/18/2021'

Loading library list...
Done


In [4]:
import json
import numpy as np

model='ff'
estwin=125
gap=0
evtwins=-30
evtwine=125
minval=50
output='df'

estwins = (estwin + gap + np.abs(evtwins))  # Estimation window start
estwine = (gap + np.abs(evtwins) + 1)       # Estimation window end
evtwinx = (estwins + 1)                     # evt time value (0=event date, -10=window start, 10=window end)
evtwins = np.abs(evtwins)                   # convert the negative to positive as we will use lag function)
evtrang = (evtwins + evtwine + 1)    

with open('data/CAR/event_samples_test.json') as data_file:
    events = json.load(data_file)
evtdata = json.dumps(events)

params = {'estwins': estwins, 'estwine': estwine, 'evtwins': evtwins, 'evtwine': evtwine, 'evtwinx': evtwinx, 'evtdata': evtdata}

df = conn.raw_sql("""
            SELECT
                    a.*,
                    x.*,
                    c.date as rdate,
                    c.ret as ret1,
                    (f.mktrf+f.rf) as mkt,
                    f.mktrf,
                    f.rf,
                    f.smb,
                    f.hml,
                    f.umd,
                    (1+c.ret)*(coalesce(d.dlret,0.00)+1)-1-(f.mktrf+f.rf) as exret,
                    (1+c.ret)*(coalesce(d.dlret,0.00)+1)-1 as ret,
                    case when c.date between a.estwin1 and a.estwin2 then 1 else 0 end as isest,
                    case when c.date between a.evtwin1 and a.evtwin2 then 1 else 0 end as isevt,
                    case
                      when c.date between a.evtwin1 and a.evtwin2 then (rank() OVER (PARTITION BY x.evtid ORDER BY c.date)-%(evtwinx)s)
                      else (rank() OVER (PARTITION BY x.evtid ORDER BY c.date))
                    end as evttime
            FROM
              (
                SELECT
                  date,
                  lag(date, %(estwins)s ) over (order by date) as estwin1,
                  lag(date, %(estwine)s )  over (order by date) as estwin2,
                  lag(date, %(evtwins)s )  over (order by date) as evtwin1,
                  lead(date, %(evtwine)s )  over (order by date) as evtwin2
                FROM crsp_a_stock.dsi
              ) as a
            JOIN
            (select
                    to_char(x.edate, 'ddMONYYYY') || trim(to_char(x.permno,'999999999')) as evtid,
                    x.permno,
                    x.edate
            from
            json_to_recordset('%(evtdata)s') as x(edate date, permno int)
            ) as x
              ON a.date=x.edate
            JOIN crsp_a_stock.dsf c
                ON x.permno=c.permno
                AND c.date BETWEEN a.estwin1 and a.evtwin2
            JOIN ff_all.factors_daily f
                ON c.date=f.date
            LEFT JOIN crsp_a_stock.dsedelist d
                ON x.permno=d.permno
                AND c.date=d.dlstdt
            WHERE f.mktrf is not null
            AND c.ret is not null
            ORDER BY x.evtid, x.permno, a.date, c.date
            """ % params)

In [ ]:
conn.raw_sql("""
            SELECT
                    a.*,
                    x.*,
                    c.date as rdate,
                    c.ret as ret1,
                    (f.mktrf+f.rf) as mkt,
                    f.mktrf,
                    f.rf,
                    f.smb,
                    f.hml,
                    f.umd,
                    (1+c.ret)*(coalesce(d.dlret,0.00)+1)-1-(f.mktrf+f.rf) as exret,
                    (1+c.ret)*(coalesce(d.dlret,0.00)+1)-1 as ret,
                    case when c.date between a.estwin1 and a.estwin2 then 1 else 0 end as isest,
                    case when c.date between a.evtwin1 and a.evtwin2 then 1 else 0 end as isevt,
                    case
                      when c.date between a.evtwin1 and a.evtwin2 then (rank() OVER (PARTITION BY x.evtid ORDER BY c.date)-%(evtwinx)s)
                      else (rank() OVER (PARTITION BY x.evtid ORDER BY c.date))
                    end as evttime
            FROM
              (
                SELECT
                  date,
                  lag(date, %(estwins)s ) over (order by date) as estwin1,
                  lag(date, %(estwine)s )  over (order by date) as estwin2,
                  lag(date, %(evtwins)s )  over (order by date) as evtwin1,
                  lead(date, %(evtwine)s )  over (order by date) as evtwin2
                FROM crsp_a_stock.dsi
              ) as a
              """ % params)

In [ ]:
conn.raw_sql("""SELECT
                  date,
                  lag(date, %(estwins)s ) over (order by date) as estwin1,
                  lag(date, %(estwine)s )  over (order by date) as estwin2,
                  lag(date, %(evtwins)s )  over (order by date) as evtwin1,
                  lead(date, %(evtwine)s )  over (order by date) as evtwin2
                FROM crsp_a_stock.dsi""" % params)

In [8]:
conn.raw_sql("""SELECT
                  max(date)
                FROM crsp_m_stock.dsf""" % params)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "crsp_d_stock.dsf" does not exist
LINE 3:                 FROM crsp_d_stock.dsf
                             ^

[SQL: SELECT
                  max(date)
                FROM crsp_d_stock.dsf]
(Background on this error at: http://sqlalche.me/e/13/f405)